In [110]:
!wget "https://www.csee.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu"
!pip install conllu
import conllu

--2022-12-11 07:38:48--  https://www.csee.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu
Resolving www.csee.umbc.edu (www.csee.umbc.edu)... 23.185.0.4, 2620:12a:8000::4, 2620:12a:8001::4
Connecting to www.csee.umbc.edu (www.csee.umbc.edu)|23.185.0.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://redirect.cs.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu [following]
--2022-12-11 07:38:48--  https://redirect.cs.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu
Resolving redirect.cs.umbc.edu (redirect.cs.umbc.edu)... 130.85.36.80
Connecting to redirect.cs.umbc.edu (redirect.cs.umbc.edu)|130.85.36.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13301116 (13M) [text/plain]
Saving to: ‘en_ewt-ud-train.conllu.8’

en_ewt-ud-train.con 100%[===================>]  12.68M  77.0MB/s    in 0.2

In [111]:
from collections import Counter, defaultdict
import numpy as np
import torch
import torch.nn as nn

In [112]:
!pip install conllu
# https://github.com/EmilStenstrom/conllu/
import conllu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
!wget "https://www.csee.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu"
with open('en_ewt-ud-train.conllu') as data:
  traindata = [i for i in conllu.parse_incr(data)]
print(len(traindata))
traindata[0][0]

            
   
#returns : 
#TokenList<Al, -, Zaman, :, American, forces, killed, Shaikh, Abdullah, al, -, Ani, ,, the, preacher, at, the, mosque, in, the, town, of, Qaim, ,, near, the, Syrian, border, ., metadata={newdoc id: "weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000", sent_id: "weblog-juancole.com_juancole_20051126063000_ENG_20051126_063000-0001", text: "Al-Zaman : American forces killed Shaikh Abdullah al-Ani, the preacher at the mosque in the town of Qaim, near the Syrian border."}>
        

--2022-12-11 07:38:56--  https://www.csee.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu
Resolving www.csee.umbc.edu (www.csee.umbc.edu)... 23.185.0.4, 2620:12a:8000::4, 2620:12a:8001::4
Connecting to www.csee.umbc.edu (www.csee.umbc.edu)|23.185.0.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://redirect.cs.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu [following]
--2022-12-11 07:38:56--  https://redirect.cs.umbc.edu/courses/undergraduate/473/f22/materials/a1/data/UD_English-EWT/en_ewt-ud-train.conllu
Resolving redirect.cs.umbc.edu (redirect.cs.umbc.edu)... 130.85.36.80
Connecting to redirect.cs.umbc.edu (redirect.cs.umbc.edu)|130.85.36.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13301116 (13M) [text/plain]
Saving to: ‘en_ewt-ud-train.conllu.9’

en_ewt-ud-train.con 100%[===================>]  12.68M  61.7MB/s    in 0.2

{'id': 1,
 'form': 'Al',
 'lemma': 'Al',
 'upos': 'PROPN',
 'xpos': 'NNP',
 'feats': {'Number': 'Sing'},
 'head': 0,
 'deprel': 'root',
 'deps': [('root', 0)],
 'misc': {'SpaceAfter': 'No'}}

In [117]:
word_map={}
def form_label(traindata):# creating the map of unique labels with their upos and arc labels
    dummylist = []   
    counter=0
    for inst in traindata:             ####################concatenating words into dictionary
      for k in inst:
         if k['form'] not in dummylist:
            dummylist.append(k['form'])
         if k['upos'] not in dummylist:
            dummylist.append(k['upos'])
         if k['deprel'] not in dummylist:
            dummylist.append(k['deprel'])               
    for i in range(len(dummylist)):
       word_map[dummylist[i]] = counter
       counter+=1

form_label(traindata)

In [114]:
import sys
class sentencestack(object):  # define class for making object for each sentence
    def __init__(self, zipped, arc_label,pos_label):
        self.stack = []
        self.buffer = []
        self.dependencylist = []
        self.zipped = zipped
        self.arc_label=arc_label
        self.pos_label = pos_label


In [115]:
#create stack object for each sentence in the training dataset  
#buffer=[]
#dependency_relation =[]
#final_stacklist=[]
#final_transitlist=[]


#building parse step

def Parse(data):
  list_of_objects =[]
  for i in data:   #define sentence [0] == root and deps=-1 intially for every sentence
    arc_label ={}
    pos_label ={}
    sentence =["root"]
    deps =[-1]
    for word in i:
       sentence.append(word['form'])
       deps.append(word['head'])
       arc_label[word['form']]=word['deprel'] # setting dictionary within each object which depicts the word with corresponding upos and arc label
       pos_label[word['form']] = word['upos']  
    zipped=zip(sentence,deps)  
    objectstack= sentencestack(zipped, arc_label,pos_label) 
    list_of_objects.append(objectstack)  
  return list_of_objects   

def start_parse(list_of_objects):  # returns the list of encodings of the sentences and their corresponding labels for training for a single batch
  tr_batch=[]
  tr_label=[]
  for obj in list_of_objects:
    list1,labels = actual_parse(obj) 
    for i in range(len(list1)): #######################list1 is a list of stacks and labels consists of corresponding stack labels
      ftlabel = []
      ftdeprel=[]
      for p in list1[i]:        ############################form a list of words and their corresponding arc and upos label in a ftvector list 
        if p != "root":
          ftlabel.append(obj.arc_label[p]) 
          ftdeprel.append(obj.pos_label[p])
      
      st_labels = ' '.join(ftlabel)    
      st_words = ' '.join(list1[i])
      st_pos = ' '.join(ftdeprel)
      finalstr = st_words +" "+ st_labels +" "+st_pos
                                     #ftvector = form_embeddings(ftvector)  ######################### form a feature vector of a single stack from stacklist
      tr_batch.append(finalstr)
      tr_label.append(labels[i])
  
  # for t in range(len(tr_label)):      #######################################encode traning labels 
  #   tr_label[t] = final_train_labels[tr_label[t]]

  return tr_batch, tr_label ####################return encoding of the all the parse sentences states with their corresponding labels for training    
 
  
def actual_parse(s):   # arc-eager parser, parsing every sentence
      obj =s
      index=0
      final_list = list(obj.zipped)
      iter=0
      item_dict={}
      item_list=[]
      transitval =""
      dummylist=[]
      transit=[]
      for i in final_list:      # add words to buffer
        obj.buffer.append(i[0])
        item_list.append(i[0])

      for i in final_list:
        item_dict[i[0]] = i[1]   # make dictionary with the values of head corresponding to each word
      
      lenbuffer = len(obj.buffer)

      while(len(obj.buffer)>0 ):
        iter+=1               
        if (len(obj.buffer)==0):
          break
        if(iter > 20):  # break loop to prevent infinite loop in certain cases
          break  
        
        if obj.buffer[0] == "root" : # check if buffer[iter] == root,append it to the stack and pop from buffer
          obj.stack.append("root")
          obj.buffer.pop(0)
          dummylist.append(list(obj.stack))
          transit.append("S")
         
        else:
          word1 = obj.stack[-1]
          index1= item_dict[word1]
          if index1 != None:
            head1 = item_list[index1]
          else:
            head1= None  
          word = obj.buffer[0]
          index = item_dict[word]
          if index != None:
            head = item_list[index]
          else:
            head = None  
          
          if (obj.buffer[0] == head1 and obj.stack[-1]!="root"):
            transitval = "LA"
            
          if (obj.stack[-1] == head):
            transitval ="RA"

          if(obj.stack[-1] != head and obj.buffer[0] !=head1):
            transitval = "S"

        if (transitval =="RA"):
            obj.stack.append(obj.buffer[0]) 
            obj.buffer.pop(0)
            dummylist.append(list(obj.stack))
            transit.append("RA")
        
        elif (transitval == "LA"):  
            obj.stack.pop(-1)
            dummylist.append(list(obj.stack))
            transit.append("LA")
        
        elif (transitval == "S"):
            obj.stack.append(obj.buffer[0]) 
            obj.buffer.pop(0)
            dummylist.append(list(obj.stack))
            transit.append("S")  
        if(len(obj.stack)<1):    # to prevent empty stack 
          break 
      return dummylist,transit       
     
#Parse(traindata)



  

In [116]:
# Training the model over traindataset
import math
import numpy
n_features = 0
def return_train_data(traindata):
      sentence_object = Parse(traindata)     #########################Parse each sentence for this batch ####################################
      train_data, train_labels = start_parse(sentence_object) ####################### a list of feature embeddings of stack states amd their correspondig label 
      print(len(train_data))
      print(len(train_labels))
      print("training started")
      #n_features = train_data.shape
      return train_data, train_labels
      

In [118]:
##################### define model #################################
import tensorflow as tf
import torch 
from tensorflow.keras.preprocessing.text import Tokenizer
from torch.nn.utils.rnn import pad_sequence

final_train_labels = {"LA":0,"RA":1,"S":2} 
X_train, Y_train = return_train_data(traindata)      ###################################return traindata ############################
tokenizer = Tokenizer(num_words=len(word_map))
tokenizer.fit_on_texts(X_train)
seq = tokenizer.texts_to_sequences(X_train)
print(seq[:2])
x_train_new = tf.keras.utils.pad_sequences(seq, maxlen=50)
x_train_new.shape
Y1_train=[]

for i in range(len(Y_train)):                        #############################################################convert labels to numbers
  Y1_train.append(final_train_labels[Y_train[i]] ) 

                                                    #####build a sequential model #######################################################

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(19000,500)  ,
  tf.keras.layers.LSTM(500, dropout=0.2),
  tf.keras.layers.Dense(64, activation='sigmoid'),
  #tf.keras.layers.Dropout(0.8),
  tf.keras.layers.Dense(3, activation ="softmax")
])

opt = tf.keras.optimizers.SGD(learning_rate=0.0001)
model.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train_new,np.array(Y1_train), epochs=10, batch_size=50, verbose=1)
  

194525
194525
training started
[[1], [1, 248, 1, 10]]
Epoch 1/10
3891/3891 [==============================] - 60s 15ms/step - loss: 1.2325 - accuracy: 0.3948
Epoch 2/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9559 - accuracy: 0.5777
Epoch 3/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9523 - accuracy: 0.5777
Epoch 4/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9520 - accuracy: 0.5777
Epoch 5/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9519 - accuracy: 0.5777
Epoch 6/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9518 - accuracy: 0.5777
Epoch 7/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9517 - accuracy: 0.5777
Epoch 8/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9516 - accuracy: 0.5777
Epoch 9/10
3891/3891 [==============================] - 58s 15ms/step - loss: 0.9515 - accuracy: 0.5777
Epoch 10/1